# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

# Bringing in the data
### decided to work with Nov/Dec/Jan first, will dip into others if necessary
### plan is to create a sample dataset to run PCA, GRID and KFOLDS to get a general starting point

In [1]:
# In here we will create and save our working data files
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_test = pd.read_csv("data/flights_test_raw.csv")
df_nov_dec_jan_2019 = pd.read_csv("data/flights_2019_jan_nov_dec_CLEAN.csv")
# df_july_2019 = pd.read_csv("data/flights_2019_july_CLEAN.csv")

In [3]:
df_passengers = pd.read_csv("data/passengers_nov_dec_jan_raw.csv")
df_passengers.head()

,departures_scheduled,departures_performed,payload,seats,passengers,freight,mail,distance,ramp_to_ramp,air_time,unique_carrier,airline_id,unique_carrier_name,region,carrier,carrier_name,carrier_group,carrier_group_new,origin_airport_id,origin_city_market_id,origin,origin_city_name,origin_country,origin_country_name,dest_airport_id,dest_city_market_id,dest,dest_city_name,dest_country,dest_country_name,aircraft_group,aircraft_type,aircraft_config,year,month,distance_group,class,data_source
0,75.0,75.0,6962210.0,18313.0,16390.0,483052.0,150581.0,2475.0,27877.0,24967.0,DL,19790,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,3,3,12478,31703,JFK,"New York, NY",US,United States,12892,32575,LAX,"Los Angeles, CA",US,United States,6,624,1,2018,11,5,F,DU
1,75.0,74.0,2212600.0,8140.0,6919.0,11629.0,34431.0,920.0,11684.0,10076.0,DL,19790,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,3,3,13198,33198,MCI,"Kansas City, MO",US,United States,14869,34614,SLC,"Salt Lake City, UT",US,United States,6,608,1,2018,11,2,F,DU
2,75.0,75.0,3732000.0,14923.0,13551.0,37667.0,3988.0,1310.0,15094.0,13560.0,DL,19790,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,3,3,13204,31454,MCO,"Orlando, FL",US,United States,13487,31650,MSP,"Minneapolis, MN",US,United States,6,622,1,2018,11,3,F,DU
3,75.0,75.0,2948400.0,11250.0,9127.0,3294.0,3866.0,361.0,6151.0,4149.0,AA,19805,American Airlines Inc.,D,AA,American Airlines Inc.,3,3,11057,31057,CLT,"Charlotte, NC",US,United States,10821,30852,BWI,"Baltimore, MD",US,United States,6,694,1,2018,11,1,F,DU
4,75.0,75.0,2684300.0,9600.0,7808.0,7126.0,3418.0,224.0,5421.0,3590.0,AA,19805,American Airlines Inc.,D,AA,American Airlines Inc.,3,3,11298,30194,DFW,"Dallas/Fort Worth, TX",US,United States,12266,31453,IAH,"Houston, TX",US,United States,6,698,1,2018,11,1,F,DU


In [4]:
len(df_passengers.origin.unique())

1793

In [5]:
features_list_passengers = ['passengers', 
                            'origin']
df_passengers = df_passengers.filter(items=features_list_passengers)

In [6]:
df_nov_dec_jan_2019.head()

,Unnamed: 0,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,airport_names,airline_names
0,0,2019-01-01,WN,WN,WN,2054,WN,2054,13198,MCI,"Kansas City, MO",11259,DAL,"Dallas, TX",1535,1543.0,8.0,19.0,1602.0,1720.0,3.0,1705,1723.0,18.0,0.0,0.0,N,90.0,100.0,78.0,1.0,461.0,"Kansas City, MO-KS - Kansas City International...",WN: Southwest Airlines Co.
1,1,2019-01-01,WN,WN,WN,2139,WN,2139,13198,MCI,"Kansas City, MO",11259,DAL,"Dallas, TX",1310,1316.0,6.0,6.0,1322.0,1443.0,3.0,1445,1446.0,1.0,0.0,0.0,N,95.0,90.0,81.0,1.0,461.0,"Kansas City, MO-KS - Kansas City International...",WN: Southwest Airlines Co.
2,2,2019-01-01,WN,WN,WN,1686,WN,1686,13198,MCI,"Kansas City, MO",11278,DCA,"Washington, DC",1825,1830.0,5.0,20.0,1850.0,2131.0,31.0,2140,2202.0,22.0,0.0,0.0,N,135.0,152.0,101.0,1.0,949.0,"Kansas City, MO-KS - Kansas City International...",WN: Southwest Airlines Co.
3,3,2019-01-01,WN,WN,WN,2232,WN,2232,13198,MCI,"Kansas City, MO",11278,DCA,"Washington, DC",835,833.0,-2.0,9.0,842.0,1126.0,10.0,1150,1136.0,-14.0,0.0,0.0,N,135.0,123.0,104.0,1.0,949.0,"Kansas City, MO-KS - Kansas City International...",WN: Southwest Airlines Co.
4,4,2019-01-01,WN,WN,WN,677,WN,677,13198,MCI,"Kansas City, MO",11292,DEN,"Denver, CO",1845,1854.0,9.0,22.0,1916.0,1947.0,8.0,1930,1955.0,25.0,0.0,0.0,N,105.0,121.0,91.0,1.0,533.0,"Kansas City, MO-KS - Kansas City International...",WN: Southwest Airlines Co.


In [7]:
df_test.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01 00:00:00.000,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,2020-01-01 00:00:00.000,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,2020-01-01 00:00:00.000,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,2020-01-01 00:00:00.000,WN,WN,WN,4761,WN,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,2020-01-01 00:00:00.000,WN,WN,WN,5162,WN,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333


In [8]:
# Change dates to datetime

In [9]:
# Dropping 'useless' columns

features_list_df = ['mkt_unique_carrier', 
                    'origin', 
                    'dest', 
                    'crs_dep_time',
                    'crs_arr_time', 
                    'crs_elapsed_time', 
                    'distance',
                    'arr_delay']

df_train = df_nov_dec_jan_2019.filter(items=features_list_df)

In [10]:
df_train.head()

,mkt_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay
0,WN,MCI,DAL,1535,1705,90.0,461.0,18.0
1,WN,MCI,DAL,1310,1445,95.0,461.0,1.0
2,WN,MCI,DCA,1825,2140,135.0,949.0,22.0
3,WN,MCI,DCA,835,1150,135.0,949.0,-14.0
4,WN,MCI,DEN,1845,1930,105.0,533.0,25.0


In [11]:
features_list_test = ['mkt_unique_carrier', 
                    'origin', 
                    'dest', 
                    'crs_dep_time',
                    'crs_arr_time',
                    'crs_elapsed_time', 
                    'distance']
df_test = df_test.filter(items=features_list_test)

In [12]:
df_test.head()

,mkt_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance
0,WN,ONT,SFO,1810,1945,95,363
1,WN,ONT,SFO,1150,1320,90,363
2,WN,ONT,SJC,2020,2130,70,333
3,WN,ONT,SJC,1340,1455,75,333
4,WN,ONT,SJC,915,1035,80,333


### Sample the data for faster run times

In [13]:
df_train_sample = df_train.sample(frac = 0.15)
df_test_sample = df_test.sample(frac = 0.15)

In [14]:
print(df_train_sample.shape)
print(df_test_sample.shape) # no arr_delay target column

(296049, 8)
(99083, 7)


In [15]:
print(df_passengers.shape)
print(len(df_passengers.origin.unique()))

(601479, 2)
1793


In [16]:
# make passengers column an np.array
pass_array = df_passengers['passengers'].values.astype(float)
type(pass_array)

numpy.ndarray

In [17]:
df_passengers.head()

,passengers,origin
0,16390.0,JFK
1,6919.0,MCI
2,13551.0,MCO
3,9127.0,CLT
4,7808.0,DFW


### Scale passengers column

In [18]:
# b = np.linalg.norm(pass_array)
# norm = pass_array / b
# print(norm)
# df_passengers["passengers"] = norm

In [19]:
# df_passengers.head()

In [20]:
df_passengers_merge = df_passengers.groupby(by="origin").sum()

In [21]:
df_passengers_merge.reset_index(inplace=True)
df_passengers_merge.head()

,origin,passengers
0,06A,15.0
1,09A,0.0
2,1B1,8.0
3,1CA,8.0
4,1G4,26225.0


## merge passengers onto train set before PCA - maybe passengers plays a role?

In [22]:
df_train_sample_withpass = df_train_sample.copy()

In [23]:
df_train_sample_withpass = df_train_sample_withpass.merge(df_passengers_merge, how='left', on='origin')
df_test_sample = df_test_sample.merge(df_passengers_merge, how='left', on='origin')

In [24]:
df_train_sample_withpass.head() # here's our train_test_split

,mkt_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,passengers
0,DL,CVG,DFW,710,858,168.0,812.0,-26.0,4252757.0
1,WN,DAL,PHX,1940,2120,160.0,879.0,14.0,9835988.0
2,AA,BMI,ORD,610,715,65.0,116.0,-14.0,222573.0
3,DL,VLD,ATL,1200,1320,80.0,208.0,-5.0,51563.0
4,WN,DEN,OAK,2205,2345,160.0,957.0,16.0,35506258.0


In [25]:
df_test_sample.head() # final test set

,mkt_unique_carrier,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,passengers
0,UA,DEN,HDN,1900,2010,70,141,35506258.0
1,B6,BUF,JFK,1728,1900,92,301,2646130.0
2,DL,BIS,MSP,1057,1230,93,386,356560.0
3,DL,JFK,ORD,758,1000,182,740,34749183.0
4,UA,DEN,CHS,1745,2302,197,1449,35506258.0


### Scale the data

In [26]:
# StandardScaling on X
from sklearn.preprocessing import StandardScaler
X_scale = df_train_sample_withpass[['mkt_unique_carrier', 'origin', 'dest', 'crs_dep_time',
       'crs_elapsed_time', 'distance', 'passengers']]
col_names = ['crs_dep_time', 'crs_elapsed_time','distance','passengers']
features = X_scale[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
X_scale[col_names] = features
X_scale["arr_delay"] = df_train_sample_withpass["arr_delay"] # re-add arr_delay

/Users/louisrossi/opt/anaconda3/envs/ml/lib/python3.8/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
/var/folders/wg/4yj17hrn3q9g_27l8zsxxy5m0000gn/T/ipykernel_45852/3492853998.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_scale["arr_delay"] = df_train_sample_withpass["arr_delay"] # re-add arr_delay


In [27]:
X_scale.head() # arr_delay there? Ready for dummies?

,mkt_unique_carrier,origin,dest,crs_dep_time,crs_elapsed_time,distance,passengers,arr_delay
0,DL,CVG,DFW,-1.262917,0.396729,0.074628,-0.946246,-26.0
1,WN,DAL,PHX,1.246353,0.284616,0.189988,-0.612860,14.0
2,AA,BMI,ORD,-1.466923,-1.046724,-1.123734,-1.186896,-14.0
3,DL,VLD,ATL,-0.263289,-0.836512,-0.965330,-1.197107,-5.0
4,WN,DEN,OAK,1.786969,0.284616,0.324287,0.919963,16.0


In [28]:
# Finally change arr_delay to binary
# X_scale['arr_delay'].values[X_scale['arr_delay'] <=0] = 0
# X_scale['arr_delay'].values[X_scale['arr_delay'] > 0] = 1 # SAMPLE NEEDS MINUTES

In [29]:
X_scale.head()

,mkt_unique_carrier,origin,dest,crs_dep_time,crs_elapsed_time,distance,passengers,arr_delay
0,DL,CVG,DFW,-1.262917,0.396729,0.074628,-0.946246,-26.0
1,WN,DAL,PHX,1.246353,0.284616,0.189988,-0.612860,14.0
2,AA,BMI,ORD,-1.466923,-1.046724,-1.123734,-1.186896,-14.0
3,DL,VLD,ATL,-0.263289,-0.836512,-0.965330,-1.197107,-5.0
4,WN,DEN,OAK,1.786969,0.284616,0.324287,0.919963,16.0


In [30]:
# StandardScaling on our TEST SET
from sklearn.preprocessing import StandardScaler
df_test_sample = df_test_sample[['mkt_unique_carrier', 'origin', 'dest', 'crs_dep_time',
       'crs_elapsed_time', 'distance', 'passengers']]
col_names = ['crs_dep_time', 'crs_elapsed_time', 'distance', 'passengers']
features = df_test_sample[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
df_test_sample[col_names] = features

In [31]:
df_test_sample.head() # ready for dummies?

,mkt_unique_carrier,origin,dest,crs_dep_time,crs_elapsed_time,distance,passengers
0,UA,DEN,HDN,1.175664,-0.990483,-1.078145,0.922439
1,B6,BUF,JFK,0.821508,-0.683394,-0.802064,-1.040237
2,DL,BIS,MSP,-0.560113,-0.669436,-0.655396,-1.176989
3,DL,JFK,ORD,-1.175768,0.572877,-0.044567,0.877221
4,UA,DEN,CHS,0.856512,0.782256,1.178816,0.922439


## OneHot works better than LabelEncoder

# Make dummies for TTS

In [32]:
# get_dummies for mkt_carrier, origin and dest
one_hot = pd.get_dummies(X_scale[['mkt_unique_carrier', 'origin', 'dest']])

In [33]:
# Drop original columns as it is now encoded
X_scale = X_scale.drop(['mkt_unique_carrier', 'origin', 'dest'],axis = 1)

In [34]:
# Join the encoded df
X_scale = X_scale.join(one_hot)
X_scale.head()

,crs_dep_time,crs_elapsed_time,distance,passengers,arr_delay,mkt_unique_carrier_AA,mkt_unique_carrier_AS,mkt_unique_carrier_B6,mkt_unique_carrier_DL,mkt_unique_carrier_F9,mkt_unique_carrier_G4,mkt_unique_carrier_HA,mkt_unique_carrier_NK,mkt_unique_carrier_UA,mkt_unique_carrier_WN,origin_ABE,origin_ABI,origin_ABQ,origin_ABR,origin_ABY,origin_ACT,origin_ACV,origin_ACY,origin_ADK,origin_ADQ,origin_AEX,origin_AGS,origin_ALB,origin_ALO,origin_ALW,origin_AMA,origin_ANC,origin_APN,origin_ART,origin_ASE,origin_ATL,origin_ATW,origin_ATY,origin_AUS,origin_AVL,origin_AVP,origin_AZA,origin_AZO,origin_BDL,origin_BET,origin_BFF,origin_BFL,origin_BFM,origin_BGM,origin_BGR,origin_BHM,origin_BIL,origin_BIS,origin_BJI,origin_BKG,origin_BLI,origin_BLV,origin_BMI,origin_BNA,origin_BOI,origin_BOS,origin_BPT,origin_BQK,origin_BQN,origin_BRD,origin_BRO,origin_BRW,origin_BTM,origin_BTR,origin_BTV,origin_BUF,origin_BUR,origin_BWI,origin_BZN,origin_CAE,origin_CAK,origin_CDC,origin_CDV,origin_CGI,origin_CHA,origin_CHO,origin_CHS,origin_CID,origin_CIU,origin_CKB,origin_CLE,origin_CLL,origin_CLT,origin_CMH,origin_CMI,origin_CMX,origin_CNY,origin_COD,origin_COS,origin_COU,origin_CPR,origin_CRP,origin_CRW,origin_CSG,origin_CVG,origin_CWA,origin_CYS,origin_DAB,origin_DAL,origin_DAY,origin_DBQ,origin_DCA,origin_DEN,origin_DFW,origin_DHN,origin_DIK,origin_DLH,origin_DRO,origin_DRT,origin_DSM,origin_DTW,origin_DUT,origin_DVL,origin_EAR,origin_EAT,origin_EAU,origin_ECP,origin_EGE,origin_EKO,origin_ELM,origin_ELP,origin_ERI,origin_ESC,origin_EUG,origin_EVV,origin_EWN,origin_EWR,origin_EYW,origin_FAI,origin_FAR,origin_FAT,origin_FAY,origin_FCA,origin_FLG,origin_FLL,origin_FLO,origin_FNT,origin_FSD,origin_FSM,origin_FWA,origin_GCC,origin_GCK,origin_GEG,origin_GFK,origin_GGG,origin_GJT,origin_GNV,origin_GPT,origin_GRB,origin_GRI,origin_GRK,origin_GRR,origin_GSO,origin_GSP,origin_GTF,origin_GTR,origin_GUC,origin_GUM,origin_HDN,origin_HGR,origin_HHH,origin_HIB,origin_HLN,origin_HNL,origin_HOB,origin_HOU,origin_HPN,origin_HRL,origin_HSV,origin_HTS,origin_HVN,origin_HYS,origin_IAD,origin_IAG,origin_IAH,origin_ICT,origin_IDA,origin_ILM,origin_IMT,origin_IND,origin_INL,origin_IPT,origin_ISN,origin_ISP,origin_ITH,origin_ITO,origin_JAC,origin_JAN,origin_JAX,origin_JFK,origin_JHM,origin_JLN,origin_JMS,origin_JNU,origin_KOA,origin_KTN,origin_LAN,origin_LAR,origin_LAS,origin_LAW,origin_LAX,origin_LBB,origin_LBE,origin_LBF,origin_LBL,origin_LCH,origin_LCK,origin_LEX,origin_LFT,origin_LGA,origin_LGB,origin_LIH,origin_LIT,origin_LNK,origin_LNY,origin_LRD,origin_LSE,origin_LWB,origin_LWS,origin_LYH,origin_MAF,origin_MBS,origin_MCI,origin_MCO,origin_MDT,origin_MDW,origin_MEI,origin_MEM,origin_MFE,origin_MFR,origin_MGM,origin_MHK,origin_MHT,origin_MIA,origin_MKE,origin_MKG,origin_MKK,origin_MLB,origin_MLI,origin_MLU,origin_MMH,origin_MOB,origin_MOT,origin_MQT,origin_MRY,origin_MSN,origin_MSO,origin_MSP,origin_MSY,origin_MTJ,origin_MYR,origin_OAJ,origin_OAK,origin_OGD,origin_OGG,origin_OGS,origin_OKC,origin_OMA,origin_OME,origin_ONT,origin_ORD,origin_ORF,origin_ORH,origin_OTH,origin_OTZ,origin_OWB,origin_PAE,origin_PAH,origin_PBG,origin_PBI,origin_PDX,origin_PGD,origin_PGV,origin_PHF,origin_PHL,origin_PHX,origin_PIA,origin_PIB,origin_PIE,origin_PIH,origin_PIR,origin_PIT,origin_PLN,origin_PNS,origin_PPG,origin_PQI,origin_PRC,origin_PSC,origin_PSE,origin_PSG,origin_PSM,origin_PSP,origin_PUB,origin_PUW,origin_PVD,origin_PVU,origin_PWM,origin_RAP,origin_RDD,origin_RDM,origin_RDU,origin_RFD,origin_RHI,origin_RIC,origin_RKS,origin_RNO,origin_ROA,origin_ROC,origin_ROW,origin_RST,origin_RSW,origin_SAF,origin_SAN,origin_SAT,origin_SAV,origin_SBA,origin_SBN,origin_SBP,origin_SBY,origin_SCC,origin_SCE,origin_SCK,origin_SDF,origin_SEA,origin_SFB,origin_SFO,origin_SGF,origin_SGU,origin_SHD,origin_SHV,origin_SIT,origin_SJC,origin_SJT,origin_SJU,origin_SLC,origin_SLN,origin_SMF,origin_SMX,origin_SNA,origin_SPI,origin_SPN,origin_SPS,origin_SRQ,origin_STC,origin_STL,origin_STS,origin_STT,origin_STX,o

In [35]:
# Perfect! Let's remove our target
X = X_scale.loc[:, X_scale.columns != 'arr_delay']
y = X_scale[['arr_delay']]

In [36]:
X.shape

(296049, 746)

In [37]:
y.shape

(296049, 1)

# Make dummies for final test

In [38]:
one_hot_test = pd.get_dummies(df_test_sample[['mkt_unique_carrier', 'origin', 'dest']])

In [39]:
# Drop original columns as it is now encoded
df_test_sample = df_test_sample.drop(['mkt_unique_carrier', 'origin', 'dest'],axis = 1)

In [40]:
# Join the encoded df
df_test_sample = df_test_sample.join(one_hot_test)
df_test_sample.head()

,crs_dep_time,crs_elapsed_time,distance,passengers,mkt_unique_carrier_AA,mkt_unique_carrier_AS,mkt_unique_carrier_B6,mkt_unique_carrier_DL,mkt_unique_carrier_F9,mkt_unique_carrier_G4,mkt_unique_carrier_HA,mkt_unique_carrier_NK,mkt_unique_carrier_UA,mkt_unique_carrier_WN,origin_ABE,origin_ABI,origin_ABQ,origin_ABR,origin_ABY,origin_ACT,origin_ACV,origin_ACY,origin_ADQ,origin_AEX,origin_AGS,origin_ALB,origin_ALO,origin_ALW,origin_AMA,origin_ANC,origin_APN,origin_ART,origin_ASE,origin_ATL,origin_ATW,origin_ATY,origin_AUS,origin_AVL,origin_AVP,origin_AZA,origin_AZO,origin_BDL,origin_BET,origin_BFF,origin_BFL,origin_BFM,origin_BGM,origin_BGR,origin_BHM,origin_BIL,origin_BIS,origin_BJI,origin_BLI,origin_BLV,origin_BMI,origin_BNA,origin_BOI,origin_BOS,origin_BPT,origin_BQK,origin_BQN,origin_BRD,origin_BRO,origin_BRW,origin_BTM,origin_BTR,origin_BTV,origin_BUF,origin_BUR,origin_BWI,origin_BZN,origin_CAE,origin_CAK,origin_CDC,origin_CDV,origin_CGI,origin_CHA,origin_CHO,origin_CHS,origin_CID,origin_CIU,origin_CKB,origin_CLE,origin_CLL,origin_CLT,origin_CMH,origin_CMI,origin_CMX,origin_CNY,origin_COD,origin_COS,origin_COU,origin_CPR,origin_CRP,origin_CRW,origin_CSG,origin_CVG,origin_CWA,origin_CYS,origin_DAB,origin_DAL,origin_DAY,origin_DBQ,origin_DCA,origin_DEN,origin_DFW,origin_DHN,origin_DIK,origin_DLH,origin_DRO,origin_DRT,origin_DSM,origin_DTW,origin_DVL,origin_EAR,origin_EAT,origin_EAU,origin_ECP,origin_EGE,origin_EKO,origin_ELM,origin_ELP,origin_ERI,origin_ESC,origin_EUG,origin_EVV,origin_EWN,origin_EWR,origin_EYW,origin_FAI,origin_FAR,origin_FAT,origin_FAY,origin_FCA,origin_FLG,origin_FLL,origin_FLO,origin_FNT,origin_FSD,origin_FSM,origin_FWA,origin_GCC,origin_GCK,origin_GEG,origin_GFK,origin_GGG,origin_GJT,origin_GNV,origin_GPT,origin_GRB,origin_GRI,origin_GRK,origin_GRR,origin_GSO,origin_GSP,origin_GTF,origin_GTR,origin_GUC,origin_GUM,origin_HDN,origin_HGR,origin_HHH,origin_HIB,origin_HLN,origin_HNL,origin_HOB,origin_HOU,origin_HPN,origin_HRL,origin_HSV,origin_HTS,origin_HVN,origin_HYS,origin_IAD,origin_IAG,origin_IAH,origin_ICT,origin_IDA,origin_ILM,origin_IMT,origin_IND,origin_INL,origin_IPT,origin_ISP,origin_ITH,origin_ITO,origin_JAC,origin_JAN,origin_JAX,origin_JFK,origin_JHM,origin_JLN,origin_JMS,origin_JNU,origin_KOA,origin_KTN,origin_LAN,origin_LAR,origin_LAS,origin_LAW,origin_LAX,origin_LBB,origin_LBE,origin_LBF,origin_LBL,origin_LCH,origin_LCK,origin_LEX,origin_LFT,origin_LGA,origin_LGB,origin_LIH,origin_LIT,origin_LNK,origin_LNY,origin_LRD,origin_LSE,origin_LWB,origin_LWS,origin_LYH,origin_MAF,origin_MBS,origin_MCI,origin_MCO,origin_MDT,origin_MDW,origin_MEI,origin_MEM,origin_MFE,origin_MFR,origin_MGM,origin_MHK,origin_MHT,origin_MIA,origin_MKE,origin_MKG,origin_MKK,origin_MLB,origin_MLI,origin_MLU,origin_MMH,origin_MOB,origin_MOT,origin_MQT,origin_MRY,origin_MSN,origin_MSO,origin_MSP,origin_MSY,origin_MTJ,origin_MYR,origin_OAJ,origin_OAK,origin_OGD,origin_OGG,origin_OGS,origin_OKC,origin_OMA,origin_OME,origin_ONT,origin_ORD,origin_ORF,origin_ORH,origin_OTH,origin_OTZ,origin_OWB,origin_PAE,origin_PAH,origin_PBG,origin_PBI,origin_PDX,origin_PGD,origin_PGV,origin_PHF,origin_PHL,origin_PHX,origin_PIA,origin_PIB,origin_PIE,origin_PIH,origin_PIR,origin_PIT,origin_PLN,origin_PNS,origin_PPG,origin_PQI,origin_PRC,origin_PSC,origin_PSE,origin_PSG,origin_PSM,origin_PSP,origin_PUB,origin_PUW,origin_PVD,origin_PVU,origin_PWM,origin_RAP,origin_RDD,origin_RDM,origin_RDU,origin_RFD,origin_RHI,origin_RIC,origin_RIW,origin_RKS,origin_RNO,origin_ROA,origin_ROC,origin_ROW,origin_RST,origin_RSW,origin_SAF,origin_SAN,origin_SAT,origin_SAV,origin_SBA,origin_SBN,origin_SBP,origin_SBY,origin_SCC,origin_SCE,origin_SCK,origin_SDF,origin_SEA,origin_SFB,origin_SFO,origin_SGF,origin_SGU,origin_SHD,origin_SHR,origin_SHV,origin_SIT,origin_SJC,origin_SJT,origin_SJU,origin_SLC,origin_SLN,origin_SMF,origin_SMX,origin_SNA,origin_SPI,origin_SPN,origin_SPS,origin_SRQ,origin_STC,origin_STL,origin_STS,origin_STT,origin_STX,origin_SUN,origin_SUX,origin_SWF,

# TRAIN_TEST_SPLIT

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.20, 
    random_state=42)

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [42]:
import pickle
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import xgboost as xgb
from xgboost import plot_importance
from sklearn.decomposition import PCA
import joblib
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

### Ridge regression, GridSearch, model, pickle save

In [50]:
rr = Ridge()
parameters = {'alpha':[0.1,1,5,10,50]}
# Ridge_tuned = GridSearchCV(rr,param_grid=parameters, scoring='r2', cv=10)
Ridge_tuned = GridSearchCV(rr, param_grid=parameters)

## Training...

In [51]:
Ridge_tuned.fit(X_train,y_train)
filename = 'Ridge_onehot_final.sav'
pickle.dump(Ridge_tuned, open(filename, 'wb'))

### ...

In [52]:
y_pred = Ridge_tuned.best_estimator_.predict(X_test)

In [53]:
print(r2_score(y_test,y_pred))
print(np.sqrt(mean_squared_error(y_test,y_pred)))

0.018970910234938687
46.75081499943135


### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.